In [2]:
import Markdown
using LambdaFn,LinearAlgebra
include("MatrixKit.jl")

@latex (macro with 2 methods)

In [3]:
norm₂(v::Vector)=v.^2 |> sum |> sqrt

norm₂ (generic function with 1 method)

In [12]:
function PowerMethod(A::Matrix,v::Vector,digits::Int=3,epochₘₐₓ::Int=100)
    report = "## PowerMethod\n"
    report *= matrix_form(:A,A)*" , "*matrix_form(:v,v) |> centralize
    λ = 0
    for i = 1:epochₘₐₓ
        v₁ = A*v
        λ₁ = abs.(v₁./v) |> maximum
        ϵ = λ₁-λ
        if ϵ |> abs |> @λ(round(_;digits=digits)) == 0
            return λ₁,v₁/norm₂(v₁),report
        end
        report *= "\n### Loop $i\n"
        report *= matrix_form("v_{$i}",v) |> centralize
        report *= "``λ=$λ₁,ϵ=$ϵ`` \n"
        λ,v = λ₁,v₁/norm₂(v₁)
    end
end

PowerMethod (generic function with 3 methods)

In [5]:
@latex A = [7. 3 -2;3 4 -1;-2 -1 3]

<center>$A=\begin{bmatrix}7.0&3.0&-2.0\\3.0&4.0&-1.0\\-2.0&-1.0&3.0\end{bmatrix}$</center>


In [13]:
λ,v,report = PowerMethod(A,[1;2;3]);
Markdown.parse(report)

## PowerMethod

<center>$A=\begin{bmatrix}7.0&3.0&-2.0\\3.0&4.0&-1.0\\-2.0&-1.0&3.0\end{bmatrix}$ , $v=\begin{bmatrix}1\\2\\3\end{bmatrix}$</center>

### Loop 1

<center>$v_{1}=\begin{bmatrix}1\\2\\3\end{bmatrix}$</center>$λ=7.0,ϵ=7.0$ 

### Loop 2

<center>$v_{2}=\begin{bmatrix}0.595879571531124\\0.6810052246069989\\0.4256282653793743\end{bmatrix}$</center>$λ=9.000000000000002,ϵ=2.0000000000000018$ 

### Loop 3

<center>$v_{3}=\begin{bmatrix}0.792343142713512\\0.6036900134960091\\-0.088038126968168\end{bmatrix}$</center>$λ=27.85714285714286,ϵ=18.85714285714286$ 

### Loop 4

<center>$v_{4}=\begin{bmatrix}0.8096295054269433\\0.5244344709610249\\-0.26356887071494817\end{bmatrix}$</center>$λ=11.133333333333331,ϵ=-16.72380952380953$ 

### Loop 5

<center>$v_{5}=\begin{bmatrix}0.8103116533020607\\0.49969453615843107\\-0.3061052025959237\end{bmatrix}$</center>$λ=9.926761860893597,ϵ=-1.2065714724397338$ 

### Loop 6

<center>$v_{6}=\begin{bmatrix}0.8104105159323053\\0.49309038293451934\\-0.31638057766529193\end{bmatrix}$</center>$λ=9.681546099948958,ϵ=-0.24521576094463882$ 

### Loop 7

<center>$v_{7}=\begin{bmatrix}0.8104652347751754\\0.4913835311124658\\-0.31888607459759716\end{bmatrix}$</center>$λ=9.62403964590912,ϵ=-0.05750645403983867$ 

### Loop 8

<center>$v_{8}=\begin{bmatrix}0.8104878977465579\\0.49094716103498337\\-0.3195000041910095\end{bmatrix}$</center>$λ=9.610087414163257,ϵ=-0.01395223174586313$ 

### Loop 9

<center>$v_{9}=\begin{bmatrix}0.810495595709143\\0.49083622002338506\\-0.3196508946479525\end{bmatrix}$</center>$λ=9.60666823337858,ϵ=-0.003419180784677067$ 

### Loop 10

<center>$v_{10}=\begin{bmatrix}0.8104979516697643\\0.49080812076296304\\-0.3196880650450453\end{bmatrix}$</center>$λ=9.605826913822792,ϵ=-0.0008413195557874786$ 


In [76]:
@latex v

<center>$v=\begin{bmatrix}0.8104988189663089\\0.49079923598900027\\-0.31969950642256556\end{bmatrix}$</center>


In [81]:
λ*v

3-element Vector{Float64}:
  7.785343186371937
  4.714430667100515
 -3.0709117839156956

In [78]:
A*v

3-element Vector{Float64}:
  7.785288453576294
  4.714392907277493
 -3.070895393189315